# Задание по программированию: Важность признаков

## Вы научитесь:
- обучать решающие деревья
- находить наиболее важные для них признаки

## Введение

Решающие деревья относятся к классу логических методов. Их основная идея состоит в объединении определенного количества простых решающих правил, благодаря чему итоговый алгоритм является интерпретируемым. Как следует из названия, решающее дерево представляет собой бинарное дерево, в котором каждой вершине сопоставлено некоторое правило вида "j-й признак имеет значение меньше b". В листьях этого дерева записаны числа-предсказания. Чтобы получить ответ, нужно стартовать из корня и делать переходы либо в левое, либо в правое поддерево в зависимости от того, выполняется правило из текущей вершины или нет.

Одна из особенностей решающих деревьев заключается в том, что они позволяют получать важности всех используемых признаков. Важность признака можно оценить на основе того, как сильно улучшился критерий качества благодаря использованию этого признака в вершинах дерева.

## Данные
В этом задании мы вновь рассмотрим данные о пассажирах Титаника. Будем решать на них задачу классификации, в которой по различным характеристикам пассажиров требуется предсказать, кто из них выжил после крушения корабля.

Реализация в Scikit-Learn
В библиотеке scikit-learn решающие деревья реализованы в классах sklearn.tree.DecisionTreeСlassifier (для классификации) и sklearn.tree.DecisionTreeRegressor (для регрессии). Обучение модели производится с помощью функции `fit`.

В этом задании вам также потребуется находить важность признаков. Это можно сделать, имея уже обученный классификатор:

`importances = clf.feature_importances_`

Переменная importances будет содержать массив "важностей" признаков. Индекс в этом массиве соответствует индексу признака в данных.

Стоит обратить внимание, что данные могут содержать пропуски. Pandas хранит такие значения как nan (not a number). Для того, чтобы проверить, является ли число nan'ом, можно воспользоваться функцией np.isnan.

Пример использования:

`np.isnan(X)`

## Материалы
- Подробнее про решающие деревья в sklearn http://scikit-learn.org/stable/modules/tree.html
- Работа с пропущенными значениями в pandas http://pandas.pydata.org/pandas-docs/stable/missing_data.html
- Подробнее о деревьях и их построении https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture07-trees.pdf

## Инструкция по выполнению

1. Загрузите выборку из файла titanic.csv с помощью пакета Pandas.
2. Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).
3. Обратите внимание, что признак Sex имеет строковые значения.
4. Выделите целевую переменную — она записана в столбце Survived.
5. В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст. Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки.
6. Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).
7. Вычислите важности признаков и найдите два признака с наибольшей важностью. Их названия будут ответами для данной задачи (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен).

Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, что отправляемые файлы не должны содержать перевод строки в конце. Данный нюанс является ограничением платформы Coursera. Мы работаем над тем, чтобы убрать это ограничение.

In [16]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

#### 1. Загрузите выборку из файла titanic.csv с помощью пакета Pandas.

In [8]:
data = pd.read_csv('data/titanic.csv')

In [9]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### 2. Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).
#### 3. Обратите внимание, что признак Sex имеет строковые значения.

#### 4. Выделите целевую переменную — она записана в столбце Survived.

#### 5. В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст. Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки.

In [21]:
X = data[['Pclass', 'Fare', 'Age', 'Sex', 'Survived']].dropna().drop('Survived', axis=1)
y = data[['Pclass', 'Fare', 'Age', 'Sex', 'Survived']].dropna()['Survived']

In [23]:
X['Sex'] = X['Sex'].map({'male':1, 'female':0})

In [24]:
X.head()

,Pclass,Fare,Age,Sex
0,3,7.2500,22.0,1
1,1,71.2833,38.0,0
2,3,7.9250,26.0,0
3,1,53.1000,35.0,0
4,3,8.0500,35.0,1


In [25]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

#### 6. Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).

In [26]:
clf = DecisionTreeClassifier(random_state=241)
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=241,
            splitter='best')

#### 7. Вычислите важности признаков и найдите два признака с наибольшей важностью. Их названия будут ответами для данной задачи (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен).

In [28]:
importances = clf.feature_importances_
importances

array([ 0.14000522,  0.30343647,  0.2560461 ,  0.30051221])

In [43]:
importances_dict = {X.columns[n]:importances[n] for n in range(len(X.columns))}

In [44]:
importances_pairs_sorted = sorted(importances_dict.items(), key=lambda x: -x[1])
importances_pairs_sorted

[('Fare', 0.30343646953145248),
 ('Sex', 0.30051221095823943),
 ('Age', 0.25604610097906622),
 ('Pclass', 0.1400052185312419)]

In [46]:
importances_sorted = list(map(lambda x: x[0], importances_dict_sorted))
importances_sorted

['Fare', 'Sex', 'Age', 'Pclass']

In [47]:
def write_answer(name, answer):
    with open('data/' + name + '.txt', 'w') as file:
        file.write(answer)

In [48]:
write_answer('importance_answer1', ' '.join(importances_sorted[:2]))